In [5]:
from IPython.core.display import display, HTML, Markdown
from ipywidgets import Button, HBox, VBox, widgets, Layout
import ipysheet
from tabulate import tabulate
import pandas as pd

In [6]:
def is_subseq(s, subs):
    """
    bool: verifica se subs è sottosequenza di s.
    """
    found = 0
    pos_r = 0
    while pos_r < len(s):
        if s[pos_r] == subs[found]:
            found += 1
            if found >= len(subs):
                return True
        pos_r += 1
    return False

def evaluation_format(answ, pt_green,pt_red):
    pt_blue=pt_red-pt_green
    return f"{answ}. Totalizzeresti <span style='color:green'>[{pt_green} safe pt]</span>, \
                                    <span style='color:blue'>[{pt_blue} possible pt]</span>, \
                                    <span style='color:red'>[{pt_red} out of reach pt]</span>.<br>"

"""
Legend of possible sequence types:
      "SC" :  implemented : strettamente crescente
      "ND" :  implemented : non-decrescente
      "SD" :  implemented : strettamente decrescente
      "NC" :  implemented : non-crescente
      "V"  :  implemented : V-sequence (first down then up)
      "A"  :  implemented : A-sequence (first up then down)
      "SV" :  implemented : strict V-sequence (first strict down then strict up)
      "SA" :  implemented : strict A-sequence (first strict up then strict down)
  "ZigZag" :  implemented : parte crescente e poi alterna
  "ZagZig" :  implemented : parte calante e poi alterna
"ZigZagEQ" :  implemented : parte crescente e poi alterna, due consecutivi possono essere uguali
"ZagZigEQ" :  implemented : parte calante e poi alterna, due consecutivi possono essere uguali
"132-free" : not done yet : dal mondo delle permutazioni pattern free per un infinità di problemi in FPT
"""

def is_seq_of_type(s, name_s, seq_type):
    """
    valuta se una sequenza s di nome name_s è di tipo seq_type (vedi tabella subito sopra).
    valore di ritorno (bool, NO_cert_string):
       In caso affermativo il bool ritornato come prima componente è True e la seconda componente è None.
       Altrimenti il bool è False e viene restituita una stringa che riporta una violazione puntuale alla proprietà richiesta.
    """
    if len(s) > 1:
        already_went_down = s[1] < s[0]
        already_went_up = s[1] > s[0]
    for i in range(1,len(s)):
        if s[i] < s[i-1]:
            already_went_down = True
            if seq_type=="V" and already_went_up:
                return (0,f"L'elemento {s[i]} in posizione {i+1} è minore dell'elemento {s[i-1]} in posizione {i} come anche l'elemento {s[i-2]} in posizione {i-1}.")
            if seq_type in {"SC","ND"} or (seq_type in {"ZigZag","ZigZagEQ"} and s[i]%2 == 1) or (seq_type in {"ZagZig","ZagZigEQ"} and s[i]%2 == 0):
                return (0,f"L'elemento {s[i]} in posizione {i+1} è minore dell'elemento {s[i-1]} in posizione {i}.")
        if s[i] > s[i-1]:
            already_went_up = True
            if seq_type=="A" and already_went_down:
                return (0,f"L'elemento {s[i]} in posizione {i+1} è maggiore dell'elemento {s[i-1]} in posizione {i} come anche l'elemento {s[i-2]} in posizione {i-1}.")
            if seq_type in {"SD","NC"} or (seq_type in {"ZagZig","ZagZigEQ"} and s[i]%2 == 1) or (seq_type in {"ZigZag","ZigZagEQ"} and s[i]%2 == 0):
                return (0,f"L'elemento {s[i]} in posizione {i+1} è maggiore dell'elemento {s[i-1]} in posizione {i}.")
        if s[i] == s[i-1] and seq_type in {"SC","SD","SV","SA","ZigZag","ZagZig"}:
            return (0,f"L'elemento {s[i]} in posizione {i+1} non è maggiore dell'elemento {s[i-1]} in posizione {i}.")
    return (1,None)

def is_subseq_of_type(s, name_s, subs, name_subs, subs_type, pt_green, pt_red, forced_ele_pos = None, start_banned_interval = None, end_banned_interval = None):
    """
    Verifica se subs, una sequenza di interi di nome name_subs, è sequenza di tipo subs_type (vedi tabella) e sottosequenza della sequenza s.
    Se forced_ele_pos != None è richiesto che subs contenga l'elemento s[forced_ele_pos].
    Se start_banned_interval,end_banned_interval != None è richiesto che subs eviti il sottointervallo bandito di s.
    Restituisce una stringa contenete la valutazione del certificato subs immesso dallo studente, a tale scopo i parametri pt_green e pt_red mentre pt_blue=pt_red-pt_green è fatto implicito.
    """
    submission_string = f"Hai inserito il certificato ${name_subs}={subs}$."
    submission_string += f"<br>L'istanza era data da ${name_s}={s}$.<br>"

    answ,NO_cert_string = is_seq_of_type(subs, name_subs, subs_type)
    if not answ:
        return submission_string + evaluation_format("No", pt_green,pt_red) + NO_cert_string
    if start_banned_interval != None or end_banned_interval != None:
        assert start_banned_interval != None and end_banned_interval != None
        if forced_ele_pos != None:
            assert forced_ele_pos < start_banned_interval or forced_ele_pos > end_banned_interval
            if forced_ele_pos > end_banned_interval:
                forced_ele_pos -= end_banned_interval 
        aux = s[:start_banned_interval-1] +s[end_banned_interval:]
    if not is_subseq(s, subs):
        return submission_string + evaluation_format("No", pt_green,pt_red) + "La sequenza ${name_subs}$ proposta non è sottosequenza di ${name_s}$."
    if forced_ele_pos != None:
        forced_ele_0basedpos = forced_ele_pos
        found_magic_point = False
        for guess_0basedpos_in_subs in range(len(subs)):
            if subs[guess_0basedpos_in_subs] == s[forced_ele_0basedpos]:
                if is_subseq(s[:forced_ele_0basedpos], subs[:guess_0basedpos_in_subs]) and is_subseq(s[forced_ele_0basedpos:], subs[guess_0basedpos_in_subs:]):
                    found_magic_point = False
        if not found_magic_point:
            return submission_string + evaluation_format("No", pt_green,pt_red) + "La sequenza ${name_subs}$ proposta non è sottosequenza di ${name_s}$."
        
    return submission_string + evaluation_format("Si", pt_green,pt_red)


## Esercizio \[60 pts\]
(campo minato) Ricerca del percorso migliore.

Un robot R, inizialmente situato nella cella $A1$, deve portarsi nella sua home H situata nella cella $G9$.

In [18]:
table = [["", 1, 2, 3, 4, 5, 6, 7, 8, 9], 
         ["A","R",2,3,1,1,3,4,7,"X"],
         ["B",2,1,"X",2,"X","X",7,1,2],
         ["C",4,"X",2,3,7,1,1,5,4],
         ["D",5,1,"X",4,5,"X",9,3,6],
         ["E",1,3,3,"X",3,1,1,10,8],
         ["F",1,4,"X",5,"X",3,1,8,9],
         ["G",7,5,2,"X",2,2,3,4,"H"]         
        ]
display(HTML(tabulate(table2, tablefmt='html', headers=[1, 2, 3, 4, 5, 6, 7, 8, 9])))
#print(tabulate(table, tablefmt='html'))

1,2,3,4,5,6,7,8,9
R,2,3,1,1,3,4,7,X
2,1,X,2,X,X,7,1,2
4,X,2,3,7,1,1,5,4
5,1,X,4,5,X,9,3,6
1,3,3,X,3,1,1,10,8
1,4,X,5,X,3,1,8,9
7,5,2,X,2,2,3,4,H


In [8]:
table2 = [
         ["R",2,3,1,1,3,4,7,"X"],
         [2,1,"X",2,"X","X",7,1,2],
         [4,"X",2,3,7,1,1,5,4],
         [5,1,"X",4,5,"X",9,3,6],
         [1,3,3,"X",3,1,1,10,8],
         [1,4,"X",5,"X",3,1,8,9],
         [7,5,2,"X",2,2,3,4,"H"]         
        ]

r=len(table2)
c=len(table2[0])
column_headers=["1","2","3","4","5","6","7","8","9"]
row_headers=["A","B","C","D","E","F","G"]
sheet=ipysheet.sheet(rows=r,columns=c,column_headers=column_headers, row_headers=row_headers, column_width=30)
for i in range(0,r):
    for j in range(0,c):
        ipysheet.cell(i,j,str(table2[i][j]),read_only=True)
HBox([sheet], layout={'max_width' : '1000px', 'overflow_y' : 'auto'})

In [14]:
columns=["1","2","3","4","5","6","7","8","9"]
index=pd.Index(["A","B","C","D","E","F","G"])
df=pd.DataFrame(table2,columns=columns,index=index)

print(df)

   1  2  3  4  5  6  7   8  9
A  R  2  3  1  1  3  4   7  X
B  2  1  X  2  X  X  7   1  2
C  4  X  2  3  7  1  1   5  4
D  5  1  X  4  5  X  9   3  6
E  1  3  3  X  3  1  1  10  8
F  1  4  X  5  X  3  1   8  9
G  7  5  2  X  2  2  3   4  H


In [28]:
#print((tabulate(df, headers='keys', tablefmt='grid')))
display(Markdown(df.to_markdown()))

|    | 1   | 2   | 3   | 4   | 5   | 6   |   7 |   8 | 9   |
|:---|:----|:----|:----|:----|:----|:----|----:|----:|:----|
| A  | R   | 2   | 3   | 1   | 1   | 3   |   4 |   7 | X   |
| B  | 2   | 1   | X   | 2   | X   | X   |   7 |   1 | 2   |
| C  | 4   | X   | 2   | 3   | 7   | 1   |   1 |   5 | 4   |
| D  | 5   | 1   | X   | 4   | 5   | X   |   9 |   3 | 6   |
| E  | 1   | 3   | 3   | X   | 3   | 1   |   1 |  10 | 8   |
| F  | 1   | 4   | X   | 5   | X   | 3   |   1 |   8 | 9   |
| G  | 7   | 5   | 2   | X   | 2   | 2   |   3 |   4 | H   |

In [29]:
wsMultiple = ipysheet.sheet(ipysheet.from_dataframe(df))
wsMultiple

Sheet(cells=(Cell(choice=[], column_end=0, column_start=0, numeric_format=None, row_end=6, row_start=0, squeez…

|   | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| A | R | 2 | 3 | 1 | 1 | 3 | 4 | 7 | X |
| B | 2 | 1 | X | 2 | X | X | 7 | 1 | 2 |
| C | 4 | X | 2 | 3 | 7 | 1 | 1 | 5 | 4 |
| D | 5 | 1 | X | 4 | 5 | X | 9 | 3 | 6 |
| E | 1 | 3 | 3 | X | 3 | 1 | 1 | 10| 8 |
| F | 1 | 4 | X | 5 | X | 3 | 1 | 8 | 9 |
| G | 7 | 5 | 2 | X | 2 | 2 | 3 | 4 | H |

Ogni cella senza mina contiene un numero di monetine come rappresentato.\
Quante monetine può raccogliere al massimo il robot?

I movimenti base possibili sono il passo verso destra (ad esempio dalla cella A4 alla cella A5) ed il passo verso in basso (ad esempio dalla cella A4 alla cella B4).\
Tuttavia il robot non può visitare le celle occupate da un Pacman (indicate con la X).

__Richieste__:

1. __\[10 pts\]__ Quante monetine può raccogliere al massimo il robot, spostandosi dalla sua posizione iniziale A1 fino a H?

2. __\[10 pts\]__ E se la partenza è in D4?

3. __\[10 pts\]__ Quante sono le soluzioni ottime con partenza in D4?

4. __\[10 pts\]__ E se con partenza in A1 il robot deve giungere in F7?

5. __\[10 pts\]__ Quante sono le soluzioni ottime con arrivo in F7?

6. __\[10 pts\]__ E se con partenza in A1 e arrivo in G9 al robot viene richiesto di passare per la cella D4?